In [1]:
# Imports
import os

import bokeh
import bokeh.plotting
import numpy as np
import pandas as pd

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
# Set path to collect data files
data_path = "../data/raw/"

# Import necessary files
OG_file = os.path.join(data_path, "TARA139.og.prok.fpkm.release")
COG_file = os.path.join(data_path, "COG.funccat.txt")
NOG_file = os.path.join(data_path, "NOG.funccat.txt")

In [3]:
# Read each input data file into pandas
OG_df = pd.read_csv(OG_file, sep='\t')
COG_df = pd.read_csv(COG_file, sep='\t', encoding='cp1252', names=['OG', 'category'])
NOG_df = pd.read_csv(NOG_file, sep='\t', encoding='cp1252', names=['OG', 'category'])

In [4]:
OG_df = OG_df.shift(periods=1, axis='columns')
OG_df.iloc[:, 0] = OG_df.index

In [5]:
# Set path to collect data files
data_path = "../data/clean/"

# Import necessary files
metadata_file = os.path.join(data_path, "companion_table_W1.csv")
nutrient_file = os.path.join(data_path, "nutrient_temp_table.csv")
core_file = os.path.join(data_path, "core_annos.csv")

In [6]:
# Read each input data file into pandas
meta_df = pd.read_csv(metadata_file)
cond_df = pd.read_csv(nutrient_file)
core_df = pd.read_csv(core_file)

In [7]:
# Extract sample label, id, and location from metadata
id_df = meta_df.iloc[:,[0, 4, 11, 14, 15, 16]]
id_df = id_df.rename(columns={'Sample label [TARA_station#_environmental-feature_size-fraction]': 'label', 
                              'PANGAEA sample identifier': 'pangea_id',
                              'Marine pelagic biomes (Longhurst 2007)': 'Biome',
                              'Ocean and sea regions (IHO General Sea Areas 1953) [MRGID registered at www.marineregions.com]': 'Region',
                              'Marine pelagic biomes  (Longhurst 2007) [MRGID registered at www.marineregions.com] ': 'Province'
                             })
# Extract temperature and id info for each sample
cond_df = cond_df.iloc[:,[0, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
cond_df = cond_df.rename(columns={'PANGAEA Sample ID': 'pangea_id', 
                                  'Mean_Lat*': 'Latitude',
                                  'Mean_Long*': 'Longitude',
                                  'Mean_Depth [m]*': 'Depth (m)',
                                  'Mean_Temperature [deg C]*': 'Temperature (°C)',
                                  'Mean_Salinity [PSU]*': 'Salinity (PSU)',
                                  'Mean_Oxygen [umol/kg]*': 'Oxygen (µmol/kg)',
                                  'Mean_Nitrates[umol/L]*': 'Nitrates (µmol/L)',
                                  'NO2 [umol/L]**': 'NO2 (µmol/L)',
                                  'PO4 [umol/L]**': 'PO4 (µmol/L)',
                                 })
# Merge all relevant data into one df
id_df = id_df.merge(cond_df, how='left', on='pangea_id')

In [8]:
OG_df = pd.melt(OG_df, ["og"], var_name="label", value_name="counts")
OG_df = OG_df.rename(columns={"og": "OG"})
OG_df.head()

,OG,label,counts
0,NOG243842,TARA_100_DCM_0.22-3,0.000000e+00
1,NOG243843,TARA_100_DCM_0.22-3,0.000000e+00
2,NOG243840,TARA_100_DCM_0.22-3,0.000000e+00
3,NOG41625,TARA_100_DCM_0.22-3,6.484686e-06
4,NOG41624,TARA_100_DCM_0.22-3,3.074175e-07


In [9]:
# Add core annotations 
core_df = core_df.rename(columns={"COG": "OG"})
OG_df = OG_df.merge(core_df, how="left", on="OG")

In [13]:
OG_df = OG_df.loc[OG_df["Ocean core"] == False]

In [14]:
abundances_list = []
sample_list = []
OG_list = []
# Calculate relative abundances for each sample
for sample in OG_df.label.unique():
    total = OG_df.loc[OG_df['label'] == sample]['counts'].sum()
    abundances = OG_df.loc[OG_df['label'] == sample].groupby('OG')['counts'].sum()/total
    abundances_list.extend(abundances.values)
    sample_list.extend([sample] * len(abundances))
    OG_list.extend(list(abundances.index))

In [15]:
OG_relative_df = pd.DataFrame({
    'abundance': abundances_list,
    'label': sample_list,
    'OG': OG_list
})

In [16]:
OG_relative_df = OG_relative_df.pivot(index="label", columns=["OG"], values="abundance").fillna(0)

In [17]:
OG_relative_df = OG_relative_df.reset_index()

In [18]:
OG_relative_df = OG_relative_df.merge(id_df, how="left", on="label")

In [17]:
OG_relative_df

,label,COG0478,COG0650,COG0680,COG0833,COG0856,COG1031,COG1039,COG1059,COG1098,...,Province,Latitude,Longitude,Depth (m),Temperature (°C),Salinity (PSU),Oxygen (µmol/kg),Nitrates (µmol/L),NO2 (µmol/L),PO4 (µmol/L)
0,TARA_004_DCM_0.22-1.6,0.000002,0.000000,0.000061,0.000001,0.002015,0.002355,0.000481,0.000023,0.000013,...,(NAST-E) North Atlantic Subtropical Gyral Prov...,36.57,-6.54,38.7,16.2,36.6,NaN,NaN,NaN,NaN
1,TARA_004_SRF_0.22-1.6,0.000000,0.000000,0.000104,0.000000,0.000812,0.000795,0.000687,0.000031,0.000020,...,(NAST-E) North Atlantic Subtropical Gyral Prov...,36.55,-6.57,10.0,20.5,36.6,NaN,NaN,NaN,NaN
2,TARA_007_DCM_0.22-1.6,0.000000,0.000000,0.001473,0.000002,0.001489,0.001647,0.000316,0.000003,0.000011,...,"(MEDI) Mediterranean Sea, Black Sea Province [...",37.04,1.95,41.7,17.4,NaN,NaN,NaN,0.00,0.01
3,TARA_007_SRF_0.22-1.6,0.000000,0.000002,0.000593,0.000003,0.000575,0.000727,0.000805,0.000003,0.000000,...,"(MEDI) Mediterranean Sea, Black Sea Province [...",37.02,1.95,7.5,23.8,37.5,NaN,NaN,0.00,0.06
4,TARA_009_DCM_0.22-1.6,0.000000,0.000000,0.000076,0.000000,0.002208,0.001964,0.000271,0.000067,0.000007,...,"(MEDI) Mediterranean Sea, Black Sea Province [...",39.07,5.86,55.0,16.2,37.8,NaN,NaN,0.02,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,TARA_151_DCM_0.22-3,0.000014,0.000000,0.000017,0.000002,0.002182,0.002200,0.000617,0.000105,0.000012,...,(NAST-E) North Atlantic Subtropical Gyral Prov...,36.19,-28.88,77.6,16.8,36.2,228.5,1.6,0.01,0.01
135,TARA_151_SRF_0.22-3,0.000000,0.000000,0.000043,0.000004,0.004311,0.004646,0.000317,0.000053,0.000000,...,(NAST-E) North Atlantic Subtropical Gyral Prov...,36.16,-29.01,5.4,17.3,36.2,232.1,0.3,0.02,0.01
136,TARA_152_MES_0.22-3,0.000010,0.000000,0.000080,0.000005,0.000468,0.000621,0.000107,0.000003,0.000000,...,(NAST-E) North Atlantic Subtropical Gyral Prov...,43.74,-16.88,791.8,10.2,35.5,175.3,18.2,0.01,1.12
137,TARA_152_MIX_0.22-3,0.000002,0.000000,0.000023,0.000037,0.002962,0.003152,0.000436,0.000106,0.000078,...,(NAST-E) North Atlantic Subtropical Gyral Prov...,43.79,-16.89,23.9,14.3,36.0,239.4,3.8,0.32,0.17


In [18]:
OG_relative_df = OG_relative_df.replace(['(DCM) deep chlorophyll maximum layer (ENVO:01000326)', 
                 '(SRF) surface water layer (ENVO:00002042)',
                 '(MES) mesopelagic zone (ENVO:00000213) & marine oxygen minimum zone (ENVO:01000065)',
                 '(MES) mesopelagic zone (ENVO:00000213)',
                 '(DCM) deep chlorophyll maximum layer (ENVO:01000326) & marine oxygen minimum zone (ENVO:01000065)',
                 '(MIX) marine epipelagic mixed layer (ENVO:01000061)',
                 'Westerlies Biome',
                 'Coastal Biome ',
                 'Trades Biome',
                 'Polar Biome',
                 '(NAO) North Atlantic Ocean [MRGID:1912]',
                 '(MS) Mediterranean Sea [MRGID:1905]', 
                 '(RS) Red Sea [MRGID:4264]',
                 '(IO) Indian Ocean [MRGID:1904]',
                 '(SAO) South Atlantic Ocean [MRGID:1914]',
                 '(SO) Southern Ocean [MRGID:1907]',
                 '(SPO) South Pacific Ocean [MRGID:1910]',
                 '(NPO) North Pacific Ocean [MRGID:1908]',
                 '(NAST-E) North Atlantic Subtropical Gyral Province [MRGID:21467]',
                 '(MEDI) Mediterranean Sea, Black Sea Province [MRGID:21465]',
                 '(REDS) Red Sea, Persian Gulf Province [MRGID:21474]',
                 '(ARAB) Northwest Arabian Sea Upwelling Province [MRGID:21475]',
                 '(MONS) Indian Monsoon Gyres Province [MRGID:21471]',
                 '(ISSG) Indian South Subtropical Gyre Province [MRGID:21472]',
                 '(EAFR) Eastern Africa Coastal Province [MRGID:21473]',
                 '(BENG) Benguela Current Coastal Province [MRGID:21470]',
                 '(SATL) South Atlantic Gyral Province [MRGID:21459]',
                 '(FKLD) Southwest Atlantic Shelves Province [MRGID:21469]',
                 '(ANTA) Antarctic Province [MRGID:21502]',
                 '(CHIL) Chile-Peru Current Coastal Province [MRGID:21495]',
                 '(SPSG) South Pacific Subtropical Gyre Province, North and South [MRGID:21486]',
                 '(PEOD) Pacific Equatorial Divergence Province [MRGID:21489]',
                 '(NPST) North Pacific Subtropical and Polar Front Provinces [MRGID:21484]',
                 '(PNEC) North Pacific Equatorial Countercurrent Province [MRGID:21488]',
                 '(CAMR) Central American Coastal Province [MRGID:21494]',
                 '(GUIA) Guianas Coastal Province [MRGID:21463]',
                 '(CARB) Caribbean Province [MRGID:21466]',
                 '(GFST) Gulf Stream Province [MRGID:21454]',
                 '(NAST-W) North Atlantic Subtropical Gyral Province [MRGID:21455]'
                ], 
                ['DCM', 'SRF', 'MES/OMZ', 'MES', 'DCM/OMZ', 'MIX', 'Westerlies', 'Coastal',
                 'Trades', 'Polar', 'NAO', 'MS',  'RS', 'IO', 'SAO', 'SO', 'SPO', 'NPO', 'NAST-E',
                 'MEDI', 'REDS', 'ARAB', 'MONS', 'ISSG', 'EAFR', 'BENG', 'SATL', 'FKLD', 'ANTA', 'CHIL',
                 'SPSG', 'PEOD', 'NPST', 'PNEC', 'CAMR', 'GUIA', 'CARB', 'GFST', 'NAST-W'
                ])

In [19]:
relative_OG_file = os.path.join(data_path, "relative_OG_with_metadata.csv")
OG_relative_df.to_csv(relative_OG_file, index=False)

In [58]:
np.mean(np.var(OG_relative_df.iloc[:, 1:8393]))

1.86911464523397e-07

In [12]:
core_OG_df = OG_df.loc[OG_df["Ocean core"] == True]

In [47]:
abundances_list = []
sample_list = []
OG_list = []
# Calculate relative abundances for each sample
for sample in core_OG_df.label.unique():
    total = core_OG_df.loc[core_OG_df['label'] == sample]['counts'].sum()
    abundances = core_OG_df.loc[core_OG_df['label'] == sample].groupby('OG')['counts'].sum()/total
    abundances_list.extend(abundances.values)
    sample_list.extend([sample] * len(abundances))
    OG_list.extend(list(abundances.index))

In [48]:
core_OG_relative_df = pd.DataFrame({
    'abundance': abundances_list,
    'label': sample_list,
    'OG': OG_list
})

In [49]:
core_OG_relative_df = core_OG_relative_df.pivot(index="label", columns=["OG"], values="abundance").fillna(0)

In [50]:
core_OG_relative_df = core_OG_relative_df.reset_index()

In [51]:
core_OG_relative_df = core_OG_relative_df.merge(id_df, how="left", on="label")

In [59]:
np.mean(np.var(core_OG_relative_df.iloc[:, 1:7609]))

5.213583679088494e-09